In [1]:
import logging

logging.basicConfig(format='\033[93m%(asctime)s %(levelname)s %(name)s:\033[0m %(message)s', level=logging.INFO, datefmt='%I:%M:%S')
import warnings
import pandas as pd

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.simplefilter(action="ignore", category=pd.errors.SettingWithCopyWarning)

In [2]:
from NNTrade.source.market.client.quote_source_client_csv import QuoteSourceClientFinamCSV
qsc = QuoteSourceClientFinamCSV("./quote")

from NNTrade.source.market.quotes.quotes_source import QuoteSource
from NNTrade.source.market.cache.filecache import FileCacheQuote
qs = QuoteSource(FileCacheQuote("./cache/quote"), qsc)

from NNTrade.common.time_frame import TimeFrame
from datetime import date
from NNTrade.common.candle_col_name import INDEX, OPEN, HIGH, LOW, CLOSE, VOLUME

In [3]:
quote_d1_m1_df = qs.get("EURUSD", TimeFrame.D, date(2010,1,2), date(2015,1,1), TimeFrame.m1)
quote_d1_m1_df

,aggregated_start_date_time,open,low,high,close,volume,isLast
start_date_time,,,,,,,
2010-01-02 00:00:00,2010-01-02,1.43220,1.4322,1.43220,1.43220,0.000,0
2010-01-02 00:01:00,2010-01-02,1.43220,1.4322,1.43270,1.43270,0.000,0
2010-01-02 00:02:00,2010-01-02,1.43220,1.4322,1.43480,1.43480,0.000,0
2010-01-02 00:04:00,2010-01-02,1.43220,1.4322,1.43480,1.43290,0.000,0
2010-01-02 00:06:00,2010-01-02,1.43220,1.4322,1.43480,1.43340,0.000,0
...,...,...,...,...,...,...,...
2014-12-31 23:55:00,2014-12-31,1.21571,1.2095,1.21697,1.21030,610004.301,0
2014-12-31 23:56:00,2014-12-31,1.21571,1.2095,1.21697,1.21033,610005.859,0
2014-12-31 23:57:00,2014-12-31,1.21571,1.2095,1.21697,1.21029,610007.050,0


In [4]:
quote_d1_df = quote_d1_m1_df[quote_d1_m1_df["isLast"] == 1]
quote_d1_df.set_index("aggregated_start_date_time",inplace=True)
quote_d1_df.index.name = INDEX
quote_d1_df = quote_d1_df[[OPEN, HIGH, LOW,CLOSE, VOLUME]]
quote_d1_df

,open,high,low,close,volume
start_date_time,,,,,
2010-01-02,1.43220,1.43480,1.4322,1.43340,0.000
2010-01-03,1.43220,1.43220,1.4322,1.43220,0.000
2010-01-04,1.43250,1.44560,1.4254,1.44130,0.000
2010-01-05,1.44150,1.44840,1.4381,1.43910,0.000
2010-01-06,1.43910,1.44160,1.4281,1.44040,0.000
...,...,...,...,...,...
2014-12-26,1.22400,1.22400,1.2166,1.21742,409106.755
2014-12-27,1.21739,1.21842,1.2170,1.21777,82022.138
2014-12-29,1.21848,1.22210,1.2165,1.21714,515478.317


In [5]:
from src.future_profit_by_limits import get_profit_by_limit, direction
from src.future_profit_by_limits.constant.closing_types import ClosingType

In [6]:
profit_df = get_profit_by_limit(quote_d1_df, direction.Long, 0.02,0.01,25)
profit_df["Idx_of_close"] = pd.to_datetime(profit_df["Idx_of_close"])
profit_df["Type_of_close"] = profit_df["Type_of_close"].apply(lambda el:el.value)
profit_df["Profit"]= profit_df["Profit"].astype(float)
print(profit_df.describe())
profit_df

            Profit                   Idx_of_close  Type_of_close
count  1503.000000                           1503    1509.000000
mean     -0.000569  2012-07-28 23:42:45.269460992      -0.363154
min      -0.010000            2010-01-08 00:00:00      -1.000000
25%      -0.010000            2011-05-23 00:00:00      -1.000000
50%      -0.010000            2012-08-02 00:00:00      -1.000000
75%       0.020000            2013-10-31 00:00:00       1.000000
max       0.020000            2014-12-31 00:00:00       1.000000
std       0.013450                            NaN       0.888697


,Profit,ClosePrice,Shift_to_close,Idx_of_close,Type_of_close
start_date_time,,,,,
2010-01-02,-0.01,1.417878,18,2010-01-20,-1
2010-01-03,-0.01,1.417878,17,2010-01-20,-1
2010-01-04,-0.01,1.418175,16,2010-01-20,-1
2010-01-05,-0.01,1.427085,3,2010-01-08,-1
2010-01-06,-0.01,1.424709,14,2010-01-20,-1
...,...,...,...,...,...
2014-12-26,-0.01,1.21176,4,2014-12-31,-1
2014-12-27,NaN,NaN,24,NaT,0
2014-12-29,NaN,NaN,24,NaT,0


In [7]:
quote_d1_df.iloc[18:21]

,open,high,low,close,volume
start_date_time,,,,,
2010-01-20,1.4291,1.4305,1.4078,1.4114,0.0
2010-01-21,1.4113,1.4143,1.4025,1.4098,0.0
2010-01-22,1.4098,1.4182,1.4062,1.4155,0.0


In [8]:
desc = quote_d1_df.iloc[:20].describe()
print(desc)
open_p = quote_d1_df.iloc[1][OPEN]
print(f"OPEN: {open_p}")
print(f"HIGH: {desc[HIGH]['max']/open_p - 1}")
print(f"LOW: {desc[LOW]['min']/open_p - 1}")

            open       high       low      close  volume
count  20.000000  20.000000  20.00000  20.000000    20.0
mean    1.438285   1.442695   1.43197   1.437150     0.0
std     0.009078   0.010279   0.01112   0.010967     0.0
min     1.411300   1.414300   1.40250   1.409800     0.0
25%     1.432450   1.438950   1.42760   1.433150     0.0
50%     1.439250   1.441650   1.43315   1.439250     0.0
75%     1.441550   1.449300   1.43895   1.440625     0.0
max     1.450900   1.458000   1.44510   1.451000     0.0
OPEN: 1.4322
HIGH: 0.018014243820695386
LOW: -0.020737327188939947


In [9]:
from src.future_extrem_deviation.get_extrem import get_extrem_of, ExtreamType

In [10]:
income_extrem_df = get_extrem_of(quote_d1_df, 25,ExtreamType.High,0.02, True)
income_extrem_df

,Extrem,Idx,Shift
start_date_time,,,
2010-01-02,1.45800,2010-01-13,11
2010-01-03,1.45800,2010-01-13,10
2010-01-04,1.45800,2010-01-13,9
2010-01-05,1.45800,2010-01-13,8
2010-01-06,1.45800,2010-01-13,7
...,...,...,...
2014-12-26,1.22400,2014-12-26,0
2014-12-27,1.22210,2014-12-29,1
2014-12-29,1.22210,2014-12-29,0


In [11]:
loss_extrem_df = get_extrem_of(quote_d1_df, 25,ExtreamType.Low, 0.01, True)
loss_extrem_df

,Extrem,Idx,Shift
start_date_time,,,
2010-01-02,1.417878,2010-01-20,18
2010-01-03,1.417878,2010-01-20,17
2010-01-04,1.418175,2010-01-20,16
2010-01-05,1.427085,2010-01-08,3
2010-01-06,1.424709,2010-01-20,14
...,...,...,...
2014-12-26,1.211760,2014-12-31,4
2014-12-27,1.209500,2014-12-31,3
2014-12-29,1.209500,2014-12-31,2
